# Quick testing:

In [168]:
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

def accept_cookies(web_driver):
    accept_cookies = wd.find_element_by_xpath('/html/body/div/div[1]/div/a')
    accept_cookies.click()

# TESTING
url = 'https://www.waterstones.com/book/doctor-who-sands-of-time/justin-richards/9781849907675'


wd = webdriver.Chrome(ChromeDriverManager().install())
wd.get(url)
accept_cookies(wd)

#test = wd.get_property('data-productid').text
#print(test)
#wd.close()

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\xiaod\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache
 


# Get product URLS and basic details:

In [1]:
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from random import randint
from time import sleep, ctime


def accept_cookies(web_driver):
    accept_cookies = wd.find_element_by_xpath('/html/body/div/div[1]/div/a')
    accept_cookies.click()


def get_waterstones_links(web_driver, url, start, end):
    """Iterates through waterstones thumbnail pages to get each product URL"""
    errors = []
    print('START:', ctime())
    product_info = []

    while start <= end:
        page = url + str(start)

        try:
            web_driver.get(page)
            products = wd.find_elements_by_class_name('info-wrap') # search result list

            for product in products:
                info = {}

                info['author'] = product.find_element_by_class_name('author').text
                info['price'] = product.find_element_by_class_name('price').text
                info['book_format'] = product.find_elements_by_class_name('format')[-1].text
                info['product_link'] = product.find_element_by_tag_name('a').get_property('href')
                
                product_info.append(info)

            start += 1

        except:
            print(f'Failed on this URL: {page}')
            start += 1
            errors.append(page)
    
    df = pd.DataFrame(product_info)
    df.to_pickle(f'product_links_{start-1}')

    print('END:', ctime())


wd = webdriver.Chrome(ChromeDriverManager().install())
wd.get('https://www.waterstones.com/books/search/')

accept_cookies(wd)
get_waterstones_links(wd, 'https://www.waterstones.com/books/search/page/', 16000, 20834) # max 20834

wd.close()

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Driver [C:\Users\xiaod\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache
 
START: Tue Feb  2 23:31:02 2021
END: Wed Feb  3 06:54:29 2021


In [ ]:
len(product_info)

# Get extra details:

In [6]:
import pandas as pd
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
from time import sleep

links = pd.read_pickle('product_links_2')
links = links.product_link.tolist()

books = []
wd = webdriver.Chrome(ChromeDriverManager().install())
url = 'https://www.waterstones.com/book/doctor-who-sands-of-time/justin-richards/9781849907675'
wd.get(url)



for link in tqdm(links[:2]):
    print(link)
    wd.get(link)


    title = wd.find_element_by_class_name('book-title').text # //*[@id="scope_book_title"]
    publisher = wd.find_element_by_xpath("//span[@itemprop='publisher']").text
    ISBN = wd.find_element_by_xpath("//span[@itemprop='isbn']").text
    categories = wd.find_element_by_xpath("//*[@class='breadcrumbs span12']").text
    full_star = wd.find_elements_by_xpath("//span[@class='star-icon full']")
    half_star = wd.find_elements_by_xpath("//span[@class='star-icon half']")
    num_of_stars = len(full_star) + len(half_star)*0.5

    number_of_reviews = wd.find_elements_by_xpath("//a[@class='reviews-trigger']")
    print(len(number_of_reviews))
    
    pages = wd.find_elements_by_xpath("//span[@itemprop='numberOfPages']")
    print(len(number_of_reviews))

    weight = wd.find_elements_by_xpath("//span[@itemprop='weight']")
    print(len(number_of_reviews))

    desc = wd.find_elements_by_xpath("//*[@itemprop='description']")[0].text


    extra_information = {'booktitle':title,
        'publisher':publisher,
        'ISBN':ISBN,
        'category':categories,
        'num_of_stars':num_of_stars,
        'num_of_reviews':number_of_reviews,
        'pages':pages,
        'weight':weight,
        'desc':desc}

    books.append(extra_information)

print(books)



[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
 
[WDM] - Driver [C:\Users\xiaod\.wdm\drivers\chromedriver\win32\88.0.4324.96\chromedriver.exe] found in cache
 50%|█████     | 1/2 [00:00<00:00,  1.07it/s]0
0
0
https://www.waterstones.com/book/venemous-animals/peterson/9780395936085
100%|██████████| 2/2 [00:01<00:00,  1.15it/s]0
0
0
[{'booktitle': 'LGB Elmo Loves You (Sesame Street) (Hardback)', 'publisher': '', 'ISBN': '', 'category': "This book can be found in:\nChildren's & Teenage > Picture books > Characters", 'num_of_stars': 0.0, 'num_of_reviews': [], 'pages': [<selenium.webdriver.remote.webelement.WebElement (session="17adfa14296080e460fa793287650374", element="a52527bc-ca1f-4248-9435-f3f0662e0d8c")>, <selenium.webdriver.remote.webelement.WebElement (session="17adfa14296080e460fa793287650374", element="d62af1b1-da79-4303-b09b-021e1a6603ac")>, <selenium.webdriver.remote.webelement.WebElement (session="17adfa14296080e460fa793287650

In [ ]:
# read the 3 pickle files into pandas // df1, df2, df3
# make a list dfs = [df1, df2, df3]
# make an empty df merged_df = pd.DataFrame()
# for df in dfs:
# pd.concatenate((merged_df, df) ,index=False)

#merged_df.to_csv('name.csv', index=False, encoding='UTF-8')
#def split_url(row):
#row.split('/')

#df['new_col'] = df['url'].apply(

In [185]:
import pandas as pd
first = pd.read_pickle('product_links_first')
second = pd.read_pickle('product_links_second')
third = pd.read_pickle('product_links_third')
#dfs = [first, second, third]
df = pd.concat(dfs, ignore_index=True)

df.to_csv('all_info.',index=False)


In [187]:
test = pd.read_csv('all_info', nrows=100)

test

,author,price,book_format,product_link
0,Sarah Albee,£3.99,Hardback,https://www.waterstones.com/book/lgb-elmo-love...
1,Peterson,£11.99,Paperback,https://www.waterstones.com/book/venemous-anim...
2,Sherry Shahan,£3.99,Paperback,https://www.waterstones.com/book/the-little-bu...
3,Hal H. Harrison,£11.99,Paperback,https://www.waterstones.com/book/a-field-guide...
4,Susan Meddaugh,£3.99,Paperback,https://www.waterstones.com/book/hog-eye/susan...
...,...,...,...,...
95,Katherine Coville,£12.99,Hardback,https://www.waterstones.com/book/the-cottage-i...
96,Lourdes Beneria,£41.99,Paperback,https://www.waterstones.com/book/gender-develo...
97,Aiko Ikeo,£44.99,Paperback,https://www.waterstones.com/book/japanese-econ...
98,Albert Marrin,£22.50,Hardback,https://www.waterstones.com/book/fdr-and-the-a...


In [142]:
test['isbn'] = test['product_link'].str.split('/')
x = test['product_link'].to_list()

In [188]:
a = 'https://www.waterstones.com/book/p-is-for-potty/naomi-kleinberg/christopher-moroney/9780385383691'
b = 'https://www.waterstones.com/book/lgb-elmo-loves-you-sesame-street/sarah-albee/9780385372831'
c = [a, b]

title = a.split('/')[4]
isbn = a.split('/')[-1]
contributors = a.split('/')[5:-1]
title = title.replace('-',' ')
print(title,isbn,author)

p is for potty 9780385383691 ['naomi-kleinberg', 'christopher-moroney']


['b']


In [ ]:
['a',['a'],'a']